# Mapping Gregorio Cortez's route 


In [ ]:
import geopandas as gpd
from pyproj import CRS
import pandas as pd
import numpy as np

from geograpy import places
import re

import geopy
from geopy.geocoders import ArcGIS


import shapely
from shapely.geometry import Point
from shapely.wkt import loads
import plotly.express as px 

import contextily as cx
import xyzservices.providers as xyz
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph
import pyproj

import spacy
from spacy import displacy

import googlemaps

import locationtagger
nlp = spacy.load("en_core_web_sm")

In [ ]:
text = "elcorridodegregoriocortez.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_trained_ner")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

In [ ]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

In [ ]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

In [ ]:
df = pd.DataFrame(ents, columns=['NER'])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['coordinates'] = df['NER'].apply(geocode)
#df['geometry'] = df['NER_Places'].apply(geocode)
df

In [ ]:
gdf = gpd.tools.geocode(df.coordinates, provider='ArcGIS')
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf["lat"]=gdf['geometry'].y
gdf ["lon"] = gdf['geometry'].x

gdf

In [ ]:
px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf, text="address",
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        labels="address",
                        )

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})



fig.show()